<a href="https://colab.research.google.com/github/rgaguiar/search_biometric/blob/main/extraction_roi_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 8.6 MB/s eta 0:00:00


In [3]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
from pathlib import Path
import os



In [6]:
# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')

options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2,
                                       min_hand_detection_confidence=0.0
                                       )

detector = vision.HandLandmarker.create_from_options(options)

def coordinate(id, h, w, lm):
    cx, cy = lm.x * w, lm.y * h
    return cx, cy

def extract_hand_roi(image_path, output_path, mirror=False):
    """
    Função para extrair o ROI da impressão palmar a partir de uma imagem de mão usando MediaPipe Hands.

    Parâmetros:
    - image_path (str): Caminho para a imagem da mão.
    - output_path (str): Caminho para salvar a imagem do ROI extraído.
    - mirror (bool): Se True, espelha a imagem horizontalmente.
    """
    img = cv2.imread(str(image_path))

    if img is None:
        print(f"Erro ao carregar a imagem: {image_path}")
        return

    if mirror:
        img = cv2.flip(img, 1)

    # Convert the image to MediaPipe Image format
    image_mp = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

    # Detect hand landmarks from the input image.
    detection_result = detector.detect(image_mp)

    h, w, c = img.shape

    if detection_result.hand_landmarks:
        for i, hand_landmarks in enumerate(detection_result.hand_landmarks):
            confidence = detection_result.handedness[i][0].score
            print(f"Caminho: {image_path} - Confiança de detecção para a mão {i + 1}: {confidence}")

            # Posição dos marcos
            x5, y5 = coordinate(5, h, w, hand_landmarks[5])
            x9, y9 = coordinate(9, h, w, hand_landmarks[9])
            x13, y13 = coordinate(13, h, w, hand_landmarks[13])
            x17, y17 = coordinate(17, h, w, hand_landmarks[17])

            x_avg_5_9 = (x5 + x9) / 2
            y_avg_5_9 = (y5 + y9) / 2

            x_avg_13_17 = (x13 + x17) / 2
            y_avg_13_17 = (y13 + y17) / 2

            angle = np.arctan2(y_avg_13_17 - y_avg_5_9, x_avg_13_17 - x_avg_5_9)
            angle_deg = np.degrees(angle)

            R = cv2.getRotationMatrix2D((w // 2, h // 2), angle_deg, 1)
            img_r = cv2.warpAffine(img, R, (w, h))

            v1 = np.array([x_avg_5_9, y_avg_5_9, 1])
            v2 = np.array([x_avg_13_17, y_avg_13_17, 1])
            v1_rot = (R[:, :2] @ v1[:2] + R[:, 2]).astype(int)
            v2_rot = (R[:, :2] @ v2[:2] + R[:, 2]).astype(int)

            ux, uy = v1_rot[0], v1_rot[1]
            lx, ly = v2_rot[0], v2_rot[1] + 3 * (v2_rot[0] - v1_rot[0]) // 3

            img_c = cv2.cvtColor(img_r, cv2.COLOR_BGR2GRAY)
            img_c = cv2.cvtColor(img_c, cv2.COLOR_GRAY2BGR)

            # Verifica se as coordenadas da ROI estão dentro dos limites da imagem
            x_min = max(int(min(lx, ux)), 0)
            x_max = min(int(max(lx, ux)), w)
            y_min = max(int(min(ly, uy)), 0)
            y_max = min(int(max(ly, uy)), h)

            # Verifica se a ROI é válida
            if x_max <= x_min or y_max <= y_min:
                print(f"ROI inválida para a imagem: {image_path}. Coordenadas: x_min={x_min}, x_max={x_max}, y_min={y_min}, y_max={y_max}")
                continue

            roi = img_c[y_min:y_max, x_min:x_max]

            # Verifica se a ROI não está vazia
            if roi.size == 0:
                print(f"ROI vazia para a imagem: {image_path}")
                continue

            # Salva a ROI
            cv2.imwrite(str(output_path), roi)

            # Limpeza explícita da memória
            del img, img_r, img_c, roi, R
            break
    else:
        print(f"Mão não detectada na imagem: {image_path}")

def process_all_folders_sequentially(input_folders, output_base_path):
    """
    Função para processar todas as subpastas de maneira sequencial, uma por vez.

    Parâmetros:
    - input_folders (list): Lista de caminhos para as pastas de entrada.
    - output_base_path (str): Caminho base para salvar os ROIs extraídos.
    """
    for input_folder in input_folders:
        for root, dirs, files in os.walk(input_folder):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = Path(root) / file
                    relative_path = Path(root).relative_to(input_folder)
                    output_folder = output_base_path / relative_path
                    output_folder.mkdir(parents=True, exist_ok=True)

                    output_path = output_folder / f'roi_{file}'
                    mirror = 'R' in os.path.basename(root)  # Verifica se o nome da pasta contém a letra 'R'
                    extract_hand_roi(image_path, output_path, mirror=mirror)  # Processa uma imagem por vez

                    # Liberação explícita da memória para cada imagem processada
                    del image_path, relative_path, output_folder, output_path

# Lista de pastas principais contendo subpastas com as imagens
input_folders = [
    Path("/content/drive/MyDrive/003.Mestrado_UFU/projeto_pesquisa_palmprint/dados_pesquisa/ChildPalmDB_1/")
]

# Pasta de saída onde os ROIs serão salvos
output_base_path = Path("/content/drive/MyDrive/003.Mestrado_UFU/projeto_pesquisa_palmprint/dados_pesquisa/ChildPalmDB_rois/v5_ChildPalmDB_1_roi")

# Processar todas as pastas de maneira sequencial
process_all_folders_sequentially(input_folders, output_base_path)

TypeError: HandLandmarkerOptions.__init__() got an unexpected keyword argument 'model_complexity'